In [1]:
import pandas as pd
import itertools 

In [2]:
crimes = pd.read_csv("binarized.csv")

In [3]:
crimes.head(n=5)

,Unnamed: 0,State,Year,Murder,Rape,Foeticide,Kidnapping and Abduction,Abetment of suicide,Exposure and abandonment,Procuration of minor girls,Selling of girls for prostitution,Prohibition of child marriage act,Other Crimes
0,0,andhra pradesh,2001,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
1,1,andhra pradesh,2001,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0
2,2,andhra pradesh,2001,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
3,3,andhra pradesh,2001,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
4,4,andhra pradesh,2001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0


In [4]:
vert_data = {}
for i in range(3,13):
    temp = []
    for j in range(len(crimes)):
       if(crimes.iloc[j][i]==1.0):
           temp.append(j)
    print(len(temp))
    vert_data[tuple([i-3])]=temp  

1187
1240
447
1185
323
666
429
154
528
968


In [5]:
min_support = 200
one_itemsets = [[i] for i in range(0,10) if len(vert_data[tuple([i])])>=min_support]
one_itemsets

[[0], [1], [2], [3], [4], [5], [6], [8], [9]]

In [6]:
freq_itemsets = []
freq_itemsets.append(one_itemsets)
i=0
while(1):
    itemsets_n = freq_itemsets[i] 
    #print(i, itemsets_n)
    itemsets_nplusone = [] 
    for a in range(0, len(itemsets_n)): # iteration to get the first n-item set
        for b in range(a+1, len(itemsets_n)): # iteration to get the second n-item set
            x=set(itemsets_n[a]).union(set(itemsets_n[b])) #generate a set of n+1-items by merging two n-item sets
            if(len(x)==i+2): # checking if the result of merging two n-item sets result in n+1-items set
                x=list(x)
                vert = set(vert_data[tuple(itemsets_n[a])]).intersection(set(vert_data[tuple(itemsets_n[b])])) #getting all the datapoints having all the n+1 items
                if(len(vert)>=min_support):  #checking if the n+1-item set is frequent
                    itemsets_nplusone.append(x)    #appending the n+1-item set to the n+1-itemsets
                    vert_data[tuple(sorted(tuple(x)))] = vert   #adding the n+1-item set info to the vertcal data
                    
    if(len(itemsets_nplusone)==0):  #break if the n+1-itemset is empty
        break 
        
    itemsets_nplusone = set(tuple(sorted(sub)) for sub in itemsets_nplusone)  #remove duplicate list items
    itemsets_nplusone = [list(i) for i in itemsets_nplusone]   #convert tuple items to list items
    freq_itemsets.append(itemsets_nplusone)  #append frequent n+1-itemsets
    i=i+1
crime = ["Murder", "Rape", "Foeticide", "Kidnapping and Abduction", "Abetment of suicide", "Exposure and abandonment", "Procuration of minor girls", "Selling of girls for prostitution", "Prohibition of child marriage act", "Other Crimes"]
for k_itemsets in freq_itemsets:
    for itemset in k_itemsets:
        print("[", end =" ")
        for item in itemset:
            print(crime[item]+", ", end =" ")
        print("]")

[ Murder,  ]
[ Rape,  ]
[ Foeticide,  ]
[ Kidnapping and Abduction,  ]
[ Abetment of suicide,  ]
[ Exposure and abandonment,  ]
[ Procuration of minor girls,  ]
[ Prohibition of child marriage act,  ]
[ Other Crimes,  ]
[ Murder,  Kidnapping and Abduction,  ]
[ Murder,  Foeticide,  ]
[ Murder,  Prohibition of child marriage act,  ]
[ Exposure and abandonment,  Other Crimes,  ]
[ Kidnapping and Abduction,  Other Crimes,  ]
[ Rape,  Kidnapping and Abduction,  ]
[ Kidnapping and Abduction,  Prohibition of child marriage act,  ]
[ Rape,  Foeticide,  ]
[ Rape,  Prohibition of child marriage act,  ]
[ Murder,  Exposure and abandonment,  ]
[ Kidnapping and Abduction,  Exposure and abandonment,  ]
[ Murder,  Other Crimes,  ]
[ Rape,  Exposure and abandonment,  ]
[ Murder,  Rape,  ]
[ Rape,  Other Crimes,  ]
[ Murder,  Rape,  Kidnapping and Abduction,  ]


In [7]:
min_conf = 0.5
rules = []
for i in range(1,len(freq_itemsets)):
    for itemset in freq_itemsets[i]:
        for j in range(1, len(itemset)):
            listx=list(itertools.combinations(itemset, j)) #generate all subsets of length j in itemset
            for x in listx:
                conf = len(vert_data[tuple(itemset)])/len(vert_data[tuple(x)]) #computing confidence for the rule x -> y
                x=set(x)  
                y=set(itemset).difference(x) 
                if conf>=min_conf:
                    print("conf of",tuple([x,y]),"=",conf) #interesting rules
                    rules.append(tuple([x,y]))

conf of ({2}, {0}) = 0.5011185682326622
conf of ({5}, {0}) = 0.509009009009009
conf of ({0}, {1}) = 0.5197978096040438


In [8]:
#finalrules = []
for rule in rules:
    #finalrule = []
    a = rule[0]
    b=rule[1]
    for elem in a:
        print(crime[elem]+", ", end=" ")
        #finalrule.append(crime[elem])
    print("--->", end=" ")
    #finalrule.append("--->")
    for elem in b:
        print(crime[elem]+", ", end=" ")
        #finalrule.append(crime[elem])
    #finalrules.append(finalrule)
    print()

Foeticide,  ---> Murder,  
Exposure and abandonment,  ---> Murder,  
Murder,  ---> Rape,  
